# Tutorial 6: Advanced Usage - Working with SQL

In this tutorial, we will showcase a few tips and tricks that helps you more easily move between Ponder and SQL. We will be using the [MIMIC-III demo dataset](https://physionet.org/content/mimiciii-demo/1.4/) as an example dataset. The MIMIC-III Clinical Database contains deidentified health-related data of patients who stayed in an intensive care unit (ICU) at the Beth Israel Deaconess Medical Center in Boston. The demo dataset contains records for 100 patients across three tables `PATIENTS`, `ICUSTAYS`, and `ADMISSIONS`. 

### Data Definition (DDL) with SQLAlchemy

In SQL, DDL statements involve modifications to the database schema, e.g., `CREATE`, `ALTER`, `DROP`. Oftentimes, you may want to run a DDL statement alongside your analysis, either via an external query editor or through SQLAlchemy. 



In [ ]:
# Install SQLAlchemy if you don't have it already
! pip install --upgrade bigquery-sqlalchemy

In [ ]:
import os; os.chdir("..");

In [ ]:
import credential

from bigquery.sqlalchemy import URL
from sqlalchemy import create_engine
engine = create_engine(URL(
    account = credential.params["account"],
    user = credential.params["user"],
    password = credential.params["password"],
    database = credential.params["database"],
    schema = credential.params["schema"],
    warehouse = credential.params["warehouse"],
    role=credential.params["role"],
))
connection = engine.connect()

Then we can run the SQL query directly to create a new database named `MIMIC3`

In [ ]:
connection.execute("CREATE DATABASE IF NOT EXISTS MIMIC3;") 

Now if we print out the database, we can see the new table MIMIC3 added: 

In [ ]:
ret = connection.execute("SHOW DATABASES;") 

In [ ]:
import pandas
pandas.DataFrame(ret.all())

### Existing SQL DML with Ponder

In [ ]:
import os; os.chdir("..")

We will be using a few example tables for the remainder of this tutorial. You can run this python script to populate the required datasets to your database. This will populate three different tables `PATIENTS`, `ADMISSIONS`, and `ICUSTAYS` to your database. 

In [ ]:
!python populate_mimic3.py > /dev/null 2>&1

Oftentime, you may already have an existing SQL script that you've been using to join and denormalize some tables or perform some pre-aggregation or ETL before you perform your analysis. You want to reuse that SQL code while working with Ponder for the remaining analysis workflow. In this example, we show how you can feed this into the `pd.read_sql` to operate on the resulting table.

In [ ]:
import credential
import ponder.bigquery
import modin.pandas as pd
credential.params["database"] = "MIMIC3"
bigquery_con = ponder.bigquery.connect(user=credential.params["user"],password=credential.params["password"],account=credential.params["account"],role=credential.params["role"],database=credential.params["database"],schema=credential.params["schema"],warehouse=credential.params["warehouse"])
ponder.bigquery.init(bigquery_con,enable_ssl=True)

For example, we may want to use [this existing SQL query](https://mimic.mit.edu/docs/iii/tutorials/intro-to-mimic-iii/#5-patient-age-and-mortality) from the MIT MIMIC-III tutorial to jumpstart our analysis. 

In [ ]:
df = pd.read_sql('''SELECT p."subject_id", p."dob", a."hadm_id",
                    a."admittime", p."expire_flag"
                    FROM MIMIC3.PUBLIC.ADMISSIONS as a
                    INNER JOIN MIMIC3.PUBLIC.PATIENTS as p
                    ON p."subject_id" = a."subject_id"''', con = bigquery_con)

In [ ]:
df

Then we can continue using Ponder by writing pandas as always.

In [ ]:
df["dob"] = pd.to_datetime(df["dob"])
df["admittime"] = pd.to_datetime(df["admittime"])

In [ ]:
df["age"] = df["admittime"].dt.year  - df["dob"].dt.year

In [ ]:
df["age"]

### Working with multiple tables

With Ponder, you can work with multiple tables at the same time by creating different dataframes using the `read_sql` or `read_csv` command.

In [ ]:
patients = pd.read_sql("PATIENTS", con=bigquery_con)
admissions = pd.read_sql("ADMISSIONS", con=bigquery_con)

Now we can work with these two dataframes in pandas. Here, we perform the same query as the SQL query above: 
```sql
SELECT p."subject_id", p."dob", a."hadm_id",
        a."admittime", p."expire_flag"
        FROM MIMIC3.PUBLIC.ADMISSIONS as a
        INNER JOIN MIMIC3.PUBLIC.PATIENTS as p
        ON p."subject_id" = a."subject_id"
```

In [ ]:
patients.merge(admissions,on="subject_id")[["subject_id_x", "dob", "hadm_id","admittime", "expire_flag"]]